In [1]:
from sklearn.datasets import make_moons
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

X, y = make_moons(n_samples=10000, noise=0.15)
X_train, X_test = train_test_split(X, test_size=0.1, random_state=41, shuffle=True)
y_train, y_test = train_test_split(y, test_size=0.1, random_state=41, shuffle=True)

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard'
)
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [2]:
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.885
RandomForestClassifier 0.991
SVC 0.993
VotingClassifier 0.994


In [3]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=1.0, bootstrap=True, n_jobs=-1,
    oob_score=True
)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)
bag_clf.oob_score_

0.989

In [4]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.989

In [5]:
bag_clf.oob_decision_function_

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]])

random forest
배깅(페이스팅)을 적용한 결정 트리의 앙상블

In [8]:
from sklearn.ensemble import RandomForestClassifier

#n_jobs = 학습에 사용할 cpu 코어 수. -1인 경우 전체
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

y_pred = rnd_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.991

In [10]:
#위의 코드와 동일한 구조를 가짐
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features="auto", max_leaf_nodes=16),
    n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1
)

bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.991

In [11]:
#결정 트리는 일부 특성을 무시하지만 랜덤 포레스트의 경우 모든 특성을 골고루 사용하기 때문에 중요도를 측정하기 유용하다.
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris['data'], iris['target'])
for name, score in zip(iris['feature_names'], rnd_clf.feature_importances_):
    print(name, score)
#아래의 결과를 봐서 petal length와 petal width가 중요하고 sepal width는 별로 중요하지 않은 것으로 나타난다.

sepal length (cm) 0.10284203541779925
sepal width (cm) 0.023735919386420765
petal length (cm) 0.4336862618129934
petal width (cm) 0.43973578338278657
